In [4]:
import sys
print(sys.executable)

d:\Users\장우진\AppData\Local\Programs\Python\Python314\python.exe


In [5]:
import sys
!{sys.executable} -m pip -V
!{sys.executable} -m pip install -U pandas


pip 25.3 from d:\Users\장우진\AppData\Local\Programs\Python\Python314\Lib\site-packages\pip (python 3.14)

   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   ---------------------------------------  11.0/11.1 MB 63.4 MB/s eta 0:00:01
   ---------------------------------------- 11.1/11.1 MB 36.3 MB/s  0:00:00
   ---------------------------------------- 0.0/12.4 MB ? eta -:--:--
   ------- -------------------------------- 2.4/12.4 MB 65.4 MB/s eta 0:00:01
   -------------------------------- ------- 10.2/12.4 MB 24.0 MB/s eta 0:00:01
   ---------------------------------------  12.3/12.4 MB 24.0 MB/s eta 0:00:01
   ---------------------------------------- 12.4/12.4 MB 17.1 MB/s  0:00:00

   ---------------------------------------- 0/4 [pytz]
   ---------------------------------------- 0/4 [pytz]
   ---------- ----------------------------- 1/4 [tzdata]
   ---------- ----------------------------- 1/4 [tzdata]
   -------------------- ------------------- 2/4 [numpy]
   -----

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [6]:
import pandas as pd
pd.__version__

'2.3.3'

In [7]:
from pathlib import Path
import pandas as pd

OUT_DIR = Path(r"D:\Users\Qube\0. 데이터\데이터분석_최종")

# 1) 질문 CSV 찾기: qube_questions*.csv 중 첫 번째
q_candidates = sorted(OUT_DIR.glob("qube_questions.csv"))
if not q_candidates:
    raise FileNotFoundError("qube_questions.csv 를 OUT_DIR에서 찾지 못함")
q_path = q_candidates[0]

# 2) 메시지 CSV 찾기: qube_messages*.csv 우선, 없으면 decoded_messages 폴더 병합
m_candidates = sorted(OUT_DIR.glob("decoded_messages.csv"))

if m_candidates:
    m_path = m_candidates[0]
    df_messages = pd.read_csv(m_path, encoding="utf-8-sig")
else:
    msg_dir = OUT_DIR / "decoded_messages"
    parts = sorted(msg_dir.glob("*.csv"))
    if not parts:
        raise FileNotFoundError("qube_messages*.csv 도 없고 decoded_messages/*.csv 도 없음")
    df_messages = pd.concat(
        (pd.read_csv(p, encoding="utf-8-sig") for p in parts),
        ignore_index=True
    )

df_questions = pd.read_csv(q_path, encoding="utf-8-sig")

print("df_questions:", df_questions.shape, "from", q_path.name)
print("df_messages:", df_messages.shape, "from", ("qube_messages" if m_candidates else "decoded_messages folder"))
df_questions.head(2)


df_questions: (876545, 28) from qube_questions.csv
df_messages: (3800534, 12) from decoded_messages folder


,source_file,QM_QST_NO,enc_key,DomName,SubName,QstFlg,TargetFlg,student_key,RepMasterKey,RepMasterName,...,total_images,student_images,master_images,has_image,first_master_idx,followup_student_msgs,has_followup,q_date,q_date_str,class_value
0,/dir/qube-data/qube/question_data/2024/12/.ipy...,8929739,14040437.0,과학,물리학I,D,1.0,20228086.0,20280625.0,박예원,...,3,1,2,True,1.0,6,True,2024-12-18,2024-12-18,unknown
1,/dir/qube-data/qube/question_data/2024/12/.ipy...,8929738,12406826.0,국어,문학,D,1.0,20105852.0,20394503.0,김도현,...,1,1,0,True,1.0,1,True,2024-12-18,2024-12-18,2 |


In [31]:
import numpy as np

def build_qa_pairs(df_q: pd.DataFrame, df_m: pd.DataFrame) -> pd.DataFrame:
    m = df_m.copy()

    # 안전하게 타입 정리
    for c in ["msg_idx"]:
        if c in m.columns:
            m[c] = pd.to_numeric(m[c], errors="coerce")

    # 정렬
    m = m.sort_values(["QM_QST_NO", "msg_idx"], kind="mergesort")

    rows = []
    for q_no, g in m.groupby("QM_QST_NO", dropna=False):
        if pd.isna(q_no) or str(q_no).strip() == "":
            continue

        dom = g["DomName"].iloc[0] if "DomName" in g.columns else ""
        sub = g["SubName"].iloc[0] if "SubName" in g.columns else ""

        # student / master 분리
        student_msgs = g[g["speaker_role"] == "student"]
        master_msgs = g[g["speaker_role"] == "master"]

        # 텍스트 컬럼 선택
        txt_col = "qst_text_decoded" if "qst_text_decoded" in g.columns else "qst_text_raw"

        q_text = "\n".join(
            student_msgs[txt_col].fillna("").astype(str).str.strip().tolist()
        ).strip()

        a_text = "\n".join(
            master_msgs[txt_col].fillna("").astype(str).str.strip().tolist()
        ).strip()

        # 답변이 없는 케이스는 MVP에서는 제외
        if a_text == "":
            continue

        rows.append({
            "QM_QST_NO": str(q_no),
            "DomName": str(dom) if dom is not None else "",
            "SubName": str(sub) if sub is not None else "",
            "question_text": q_text,
            "answer_text": a_text,
            "has_image": bool(g["has_image"].any()) if "has_image" in g.columns else False,
        })

    qa = pd.DataFrame(rows)
    # 너무 짧은 질문 제거(노이즈)
    qa["q_len"] = qa["question_text"].fillna("").astype(str).str.len()
    qa = qa[qa["q_len"] >= 5].drop(columns=["q_len"])
    return qa.reset_index(drop=True)

qa_df = build_qa_pairs(df_questions, df_messages)
print("qa_df:", qa_df.shape)
qa_df.head(3)


qa_df: (628658, 6)


,QM_QST_NO,DomName,SubName,question_text,answer_text,has_image
0,8927612,사회,정치와법,1 이해가 안 가요\n1 이해가 안 가요\n친권을 A가 가져간거 아닌가요??\n친권...,면접교섭권은 가정법원이 직접 지정해야합니다^^\n면접교섭권은 가정법원이 직접 지정해...,True
1,8927616,국어,고등국어,?!\n?!\n손글씨 2 개용\n손글씨 2 개용\n샤 시로 나뉘는게 모음조화때문인지...,모음조화 때문에 나뉜다는게 무슨 말쓰이신지 한번만 설명해주시겠어요?\n모음조화 때문...,True
2,8927618,수학,수학II,머가 문제죠ㅜㅜ\n머가 문제죠ㅜㅜ\n저렇게 햇어요ㅜㅜㅜ 군데 이상하게 나와용우ㅜ\n...,미지수를 하신 것처럼 설정하면 식이 너무 복잡해지고 수학2에서 배운 삼차방정식 혹은...,True


In [11]:
import sys
!{sys.executable} -m pip install -U scikit-learn


   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   ----------------------- ---------------- 4.7/8.1 MB 26.0 MB/s eta 0:00:01
   -------------------------------------- - 7.9/8.1 MB 30.7 MB/s eta 0:00:01
   ---------------------------------------- 8.1/8.1 MB 13.8 MB/s  0:00:00
   ---------------------------------------- 0.0/39.4 MB ? eta -:--:--
   ------- -------------------------------- 7.3/39.4 MB 36.7 MB/s eta 0:00:01
   ---------- ----------------------------- 10.2/39.4 MB 24.8 MB/s eta 0:00:02
   ---------------- ----------------------- 16.0/39.4 MB 26.0 MB/s eta 0:00:01
   ------------------------- -------------- 24.9/39.4 MB 30.5 MB/s eta 0:00:01
   ------------------------------------- -- 36.4/39.4 MB 35.5 MB/s eta 0:00:01
   ---------------------------------------  39.3/39.4 MB 36.7 MB/s eta 0:00:01
   ---------------------------------------  39.3/39.4 MB 36.7 MB/s eta 0:00:01
   ---------------------------------------- 39.4/39.4 MB 25.6 MB/s  0:00:01

   -

In [12]:
import sklearn
print(sklearn.__version__)


1.8.0


In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def build_tfidf_index(qa: pd.DataFrame):
    # 과목 정보를 앞에 붙여 subject-aware retrieval
    corpus = (qa["DomName"].fillna("") + " " + qa["SubName"].fillna("") + "\n" + qa["question_text"].fillna("")).tolist()

    vec = TfidfVectorizer(
        max_features=200000,
        ngram_range=(1,2),
        min_df=2
    )
    X = vec.fit_transform(corpus)
    return vec, X

tfidf_vec, tfidf_X = build_tfidf_index(qa_df)
print("tfidf_X:", tfidf_X.shape)


tfidf_X: (628658, 200000)


In [15]:
import sys
!{sys.executable} -m pip install -U requests



   ---------------------------------------- 0/5 [urllib3]
   ---------------- ----------------------- 2/5 [charset_normalizer]
   ---------------- ----------------------- 2/5 [charset_normalizer]
   ---------------- ----------------------- 2/5 [charset_normalizer]
   ---------------------------------------- 5/5 [requests]



  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [16]:
import requests
print(requests.__version__)

2.32.5


In [17]:
import os, json, requests
from typing import List, Dict, Any, Optional
from urllib.parse import unquote_plus

def decode_text(s: str) -> str:
    if s is None:
        return ""
    s = str(s)
    try:
        return unquote_plus(s)
    except Exception:
        return s

class LLMClient:
    def __init__(self):
        self.base_url = os.getenv("LLM_BASE_URL", "https://api.openai.com/v1").rstrip("/")
        self.api_key = os.getenv("LLM_API_KEY", "")
        self.model = os.getenv("LLM_MODEL", "gpt-4o-mini")
        self.timeout = int(os.getenv("LLM_TIMEOUT", "60"))

    def enabled(self) -> bool:
        return bool(self.api_key)

    def chat(self, messages: List[Dict[str, Any]], temperature: float = 0.2, max_tokens: int = 800) -> str:
        if not self.enabled():
            raise RuntimeError("LLM_API_KEY not set")

        url = f"{self.base_url}/chat/completions"
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json",
        }
        payload = {
            "model": self.model,
            "messages": messages,
            "temperature": temperature,
            "max_tokens": max_tokens,
        }
        r = requests.post(url, headers=headers, data=json.dumps(payload), timeout=self.timeout)
        r.raise_for_status()
        data = r.json()
        return data["choices"][0]["message"]["content"]

llm = LLMClient()

def retrieve_similar(
    question_text: str,
    dom: str,
    sub: str,
    top_k: int = 5
) -> List[Dict[str, Any]]:
    query = f"{dom} {sub}\n{question_text}"
    qv = tfidf_vec.transform([query])
    sims = cosine_similarity(qv, tfidf_X).ravel()
    if len(sims) == 0:
        return []

    idx = np.argsort(-sims)[:top_k]
    out = []
    for i in idx:
        out.append({
            "score": float(sims[i]),
            "QM_QST_NO": qa_df.iloc[i]["QM_QST_NO"],
            "DomName": qa_df.iloc[i]["DomName"],
            "SubName": qa_df.iloc[i]["SubName"],
            "question_text": qa_df.iloc[i]["question_text"],
            "answer_text": qa_df.iloc[i]["answer_text"],
        })
    return out

def make_answer(
    question_text: str,
    dom: str = "",
    sub: str = "",
    image_urls: Optional[List[str]] = None,
    top_k: int = 4,
    sim_threshold: float = 0.18
) -> Dict[str, Any]:
    q = decode_text(question_text).strip()
    dom = (dom or "").strip()
    sub = (sub or "").strip()
    image_urls = image_urls or []

    retrieved = retrieve_similar(q, dom, sub, top_k=top_k)
    best_score = retrieved[0]["score"] if retrieved else 0.0

    # LLM 없이도 MVP가 돌아가게: 유사도가 높으면 과거 답변을 요약해서 제공
    if (not llm.enabled()) or (best_score < sim_threshold):
        if retrieved:
            base = retrieved[0]
            draft = base["answer_text"].strip()
            # 너무 길면 잘라서 MVP로만
            if len(draft) > 1600:
                draft = draft[:1600] + "\n\n(중략)"
            return {
                "mode": "retrieval_only" if not llm.enabled() else "low_confidence_retrieval",
                "best_score": best_score,
                "answer": f"유사 질문 기반 답변\n\n{draft}\n\n추가로 문제 조건, 보기, 사진을 더 주면 더 정확하게 정리 가능",
                "citations": [{"QM_QST_NO": r["QM_QST_NO"], "score": r["score"]} for r in retrieved[:3]]
            }

        return {
            "mode": "no_data",
            "best_score": 0.0,
            "answer": "유사 사례가 부족함. 문제 지문, 보기, 사진을 추가로 주면 해결 가능",
            "citations": []
        }

    # LLM 사용: Retrieval을 근거로 답변 생성
    exemplars = []
    for r in retrieved[:top_k]:
        exemplars.append(
            f"- 유사도 {r['score']:.3f} / Q {r['QM_QST_NO']} / {r['DomName']} {r['SubName']}\n"
            f"  유사 질문 요약:\n{r['question_text'][:500]}\n"
            f"  마스터 답변:\n{r['answer_text'][:900]}\n"
        )
    exemplar_block = "\n".join(exemplars)

    system = (
        "너는 메가스터디 큐브의 AI 마스터 역할.\n"
        "목표: 학생 질문에 빠르고 정확하게, 풀이 흐름을 단계적으로 설명.\n"
        "규칙:\n"
        "1) 근거가 부족하면 추가 정보 요청\n"
        "2) 과목/단원 맥락을 반영\n"
        "3) 답만 던지지 말고 핵심 개념과 이유를 짧게\n"
        "4) 사설/개인정보는 만들지 말기\n"
    )

    user_text = (
        f"과목: {dom} / {sub}\n"
        f"학생 질문:\n{q}\n\n"
        f"참고 가능한 유사 사례(마스터 답변 포함):\n{exemplar_block}\n\n"
        "요구 출력:\n"
        "- 결론 한 줄\n"
        "- 풀이 단계 3~8개\n"
        "- 마지막에 학생이 다시 틀릴 포인트 1~2개 경고\n"
    )

    # 이미지 URL이 있으면 텍스트로 첨부(비전 지원 엔드포인트면 여기 확장 가능)
    if image_urls:
        user_text += "\n\n첨부 이미지 URL:\n" + "\n".join(image_urls)

    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": user_text},
    ]

    ans = llm.chat(messages, temperature=0.2, max_tokens=900)

    return {
        "mode": "llm_with_retrieval",
        "best_score": best_score,
        "answer": ans,
        "citations": [{"QM_QST_NO": r["QM_QST_NO"], "score": r["score"]} for r in retrieved[:3]]
    }


In [19]:
import sys
!{sys.executable} -m pip install -U uvicorn fastapi

  Using cached uvicorn-0.40.0-py3-none-any.whl.metadata (6.7 kB)
  Using cached fastapi-0.128.0-py3-none-any.whl.metadata (30 kB)
  Using cached starlette-0.50.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached pydantic-2.12.5-py3-none-any.whl.metadata (90 kB)
  Using cached annotated_doc-0.0.4-py3-none-any.whl.metadata (6.6 kB)
Using cached uvicorn-0.40.0-py3-none-any.whl (68 kB)
Using cached fastapi-0.128.0-py3-none-any.whl (103 kB)
Using cached starlette-0.50.0-py3-none-any.whl (74 kB)
Using cached annotated_doc-0.0.4-py3-none-any.whl (5.3 kB)
Using cached pydantic-2.12.5-py3-none-any.whl (463 kB)
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ------------------------------------ --- 1.8/2.0 MB 11.3 MB/s eta 0:00:01
   ---------------------------------------- 2.0/2.0 MB 8.5 MB/s  0:00:00

   --- ------------------------------------  1/12 [h11]
   ------ ---------------------------------  2/12 [click]
   ---------- -----------------------------  3/12 [anyio]

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [20]:
import fastapi
print(fastapi.__version__)

0.128.0


In [26]:
import subprocess
port = 8000  # 너가 띄운 포트로 바꿔
print(subprocess.check_output(f'netstat -ano | findstr :{port}', shell=True).decode(errors="ignore"))


  TCP    0.0.0.0:8000           0.0.0.0:0              LISTENING       9620



In [27]:
import subprocess
print(subprocess.check_output("tasklist /FI \"PID eq 9620\"", shell=True).decode(errors="ignore"))



̹ ̸                    PID  ̸              #  ޸ 
========================= ======== ================ =========== ============
python.exe                    9620 Console                    1  3,521,448 K



In [28]:
import urllib.request
with urllib.request.urlopen("http://127.0.0.1:8000/", timeout=3) as r:
    print(r.status)


200


In [29]:
import subprocess, re

out = subprocess.check_output("ipconfig", shell=True).decode(errors="ignore")
# IPv4 주소만 뽑기
ips = re.findall(r"IPv4[^:]*:\s*([0-9]+\.[0-9]+\.[0-9]+\.[0-9]+)", out)
print("IPv4 candidates:", ips)


IPv4 candidates: ['10.1.0.59']


In [30]:
@app.get("/health")
def health():
    return {"ok": True}


In [ ]:
from fastapi import FastAPI
from fastapi.responses import HTMLResponse
from pydantic import BaseModel
import threading
import uvicorn
import subprocess

app = FastAPI(title="Qube Agentic AI MVP")

class AskReq(BaseModel):
    question_text: str
    dom: str = ""
    sub: str = ""
    image_urls: list[str] = []

@app.get("/", response_class=HTMLResponse)
def home():
    # 모바일 친화 단순 UI
    return HTMLResponse("""
<!doctype html>
<html>
<head>
<meta name=viewport content="width=device-width, initial-scale=1">
<title>Qube AI MVP</title>
<style>
body{font-family:system-ui;margin:0;background:#0b0f14;color:#e8eef5}
.wrap{max-width:900px;margin:0 auto;padding:16px}
.card{background:#121826;border:1px solid #1f2a3a;border-radius:14px;padding:14px;margin-bottom:12px}
.row{display:flex;gap:8px;flex-wrap:wrap}
input,textarea{width:100%;padding:10px;border-radius:10px;border:1px solid #243248;background:#0b0f14;color:#e8eef5}
button{padding:10px 14px;border-radius:12px;border:1px solid #2a3a55;background:#1b2a44;color:#e8eef5;font-weight:600}
.msg{white-space:pre-wrap;line-height:1.5}
.small{opacity:.8;font-size:12px}
</style>
</head>
<body>
<div class="wrap">
  <div class="card">
    <div class="row">
      <input id="dom" placeholder="DomName 예: 수학, 영어, 과학">
      <input id="sub" placeholder="SubName 예: 미적분, 물리학I">
    </div>
    <textarea id="q" rows="5" placeholder="질문을 붙여넣기 (url 인코딩된 텍스트도 가능)"></textarea>
    <div class="row" style="margin-top:10px">
      <button onclick="send()">질문 보내기</button>
      <div class="small" id="status"></div>
    </div>
  </div>

  <div class="card">
    <div class="small">답변</div>
    <div id="a" class="msg"></div>
    <div id="c" class="small"></div>
  </div>
</div>

<script>
async function send(){
  const dom = document.getElementById('dom').value || '';
  const sub = document.getElementById('sub').value || '';
  const q = document.getElementById('q').value || '';
  document.getElementById('status').textContent = '처리중...';
  document.getElementById('a').textContent = '';
  document.getElementById('c').textContent = '';

  const res = await fetch('/api/ask', {
    method:'POST',
    headers:{'Content-Type':'application/json'},
    body: JSON.stringify({question_text:q, dom:dom, sub:sub, image_urls:[]})
  });

  const data = await res.json();
  document.getElementById('a').textContent = data.answer || '';
  if (data.citations && data.citations.length){
    document.getElementById('c').textContent =
      '유사 사례: ' + data.citations.map(x => x.QM_QST_NO + ' (' + x.score.toFixed(3) + ')').join(', ');
  }
  document.getElementById('status').textContent = data.mode + ' / best_score ' + (data.best_score||0).toFixed(3);
}
</script>
</body>
</html>
""")

@app.post("/api/ask")
def api_ask(req: AskReq):
    return make_answer(
        question_text=req.question_text,
        dom=req.dom,
        sub=req.sub,
        image_urls=req.image_urls
    )

def print_server_ips():
    # 리눅스에서 가장 쉬움
    try:
        ips = subprocess.check_output(["hostname", "-I"]).decode().strip()
        print("Server IPs:", ips)
    except Exception as e:
        print("Could not get IP via hostname -I:", e)

def run_server(host="0.0.0.0", port=8000):
    print_server_ips()
    uvicorn.run(app, host=host, port=port, log_level="warning")

thread = threading.Thread(target=run_server, kwargs={"host":"0.0.0.0","port":8000}, daemon=True)
thread.start()

print("Open on mobile: http://서버IP:8000")


Open on mobile: http://서버IP:8000


Could not get IP via hostname -I: Command '['hostname', '-I']' returned non-zero exit status 1.


ERROR:    [Errno 10048] error while attempting to bind on address ('0.0.0.0', 8000): [winerror 10048] 각 소켓 주소(프로토콜/네트워크 주소/포트)는 하나만 사용할 수 있습니다
